In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

from huggingface_hub import login
login(token="")

In [ ]:
import pandas as pd
# baseline_path="complete_testing_data/SAMPLE_combined_unseen_data.csv"
baseline_path = 'Sythetic_old_question.csv'
baseline = pd.read_csv(baseline_path)

In [2]:
baseline

,Title,Accepted Answer Body,Catalog
0,How to scrape a webpage url which is same as m...,You need to make a selection from the Fiscal Y...,PYTHON
1,Change some input-fields in a PDF?,I was able to solve this using pymupdf \n impo...,PYTHON
2,Role-based authentication not working with Key...,hasRole('user') requires the ROLE_user auth...,JAVA
3,Get GridPane to properly fill window in JavaFX,"Shape classes, such as Rectangle , are not r...",JAVA
4,Timing issue with Kivy,You are running the data_routines() on the m...,PYTHON
...,...,...,...
380,Search and Replace in a string with multiple p...,If you really wanted to make only one pass thr...,JAVA
381,Stream map returns null instead of stream,According to JavaDoc this is expected behavi...,JAVA
382,Appropriate applications for the as_strided fu...,"Assuming that you really, in reality, want a 3...",PYTHON
383,find duplicate more than x times in 3 dimensio...,Here is an approach using pandas . I've round...,PYTHON


In [3]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import torch
import json
from tqdm import tqdm
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
from nltk.tokenize import sent_tokenize
device = "cuda" if torch.cuda.is_available() else "cpu"
# Download the Punkt Tokenizer Models (only need to do this once)
nltk.download('punkt')

/home/21fl11/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-02-25 01:00:58.059446: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-25 01:00:58.079427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740445258.110211  774740 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740445258.12

True

In [4]:
import random
import os

In [5]:

encoder_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to(device)

In [6]:
# import _pickle as cPickle
# import pickle
# with open('../CODE_POST_OVERALL-EMBEDDINGS_DATA_V3.pkl', 'rb') as f:
#     OVO_data = cPickle.load(f)

In [7]:
import numpy as np
embeddings_path = "../AnswerEmbedding/all_embeddings.npy"
# embeddings_path = "../AnswerEmbedding/all_embeddings.pt"
sentences_path = "../AnswerEmbedding/all_sentences.npy"

sentence_embedding = np.load(embeddings_path, allow_pickle=True)
# sentence_embedding = torch.load(embeddings_path)
sentence_context = np.load(sentences_path, allow_pickle=True)

In [8]:
import faiss
def normalize_embeddings(embeddings):
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

# Assuming tensor_embedding is a NumPy array of shape (N, D)
tensor_embedding_np = normalize_embeddings(sentence_embedding)  # Normalize

# Create FAISS index
index = faiss.IndexFlatIP(tensor_embedding_np.shape[1])  # Inner Product (IP) for cosine similarity
index.add(tensor_embedding_np)

In [12]:
def build_hypothetic_prompt(question):
    prompt_template = """
    Question: {}
    """.format(question)

    return prompt_template

sys_prompt = """ 
For entire output, you are restrict to generate only ONE sentence within 10 words related to the question in one line. Don't include any other thing except ONE sentence I ask you to generate. """

In [13]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Define the model ID
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.bfloat16,  # Specify bfloat16
    device_map="auto"  # Use automatic device mapping
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, StoppingCriteria, StoppingCriteriaList

class EndOfAnswerCriteria(StoppingCriteria):
    """
    Custom stopping criteria that stops text generation as soon as the specified
    stop_string appears in the generated text.
    """
    def __init__(self, stop_string: str, tokenizer):
        self.stop_string = stop_string
        self.tokenizer = tokenizer

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # Decode the current text
        decoded = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
        # Return True to stop if the stop_string is found
        return self.stop_string in decoded

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<PAD>'})

model.resize_token_embeddings(len(tokenizer))

Embedding(128257, 4096)

In [15]:
stop_token = "END_OF_ANSWER"

In [16]:
Hy_prompt_list = []
for i in range(len(baseline)):
    # question = baseline.iloc[i]['Paraphrased Question']
    question = baseline.iloc[i]['Title']
    output_prompt = build_hypothetic_prompt(question)
    Hy_prompt_list.append(output_prompt)

In [17]:
# Create a pipeline
pipeline = transformers.pipeline(
    task="text-generation",
    model=model,  # Use the model loaded above
    tokenizer=tokenizer,  # Use the tokenizer loaded above
    model_kwargs={"torch_dtype": torch.bfloat16},  # Pass torch_dtype for precision
    device_map="auto",  # Ensure device allocation is automatic
    max_new_tokens=20,  # Limit the number of tokens in the output
    temperature=0.7,
    repetition_penalty=1.1,  # Avoid repetition
    pad_token_id=tokenizer.eos_token_id,  # Avoid padding-related errors
    return_full_text=False
)

In [18]:
from tqdm import tqdm
# Define the system prompt

# Define your user prompts
Hy_response_list = []

# Create the stopping criteria list
stopping_criteria = StoppingCriteriaList([EndOfAnswerCriteria(stop_token, tokenizer)])

# Generate answers for each prompt
for user_prompt in tqdm(Hy_prompt_list):
    # Combine the system prompt with the user prompt
    final_prompt = f"<<SYS>>\n{sys_prompt}\n<</SYS>>\n\n{user_prompt}"

    # Call the pipeline with the final prompt and the stopping criteria
    outputs = pipeline(final_prompt, stopping_criteria=stopping_criteria)
    result = outputs[0]["generated_text"]

    # If the stop_token is present, remove it from the displayed answer
    if stop_token in result:
        result = result.split(stop_token)[0].strip()
    Hy_response_list.append(result)
    print(f"Answer: {result}\n{'-'*50}")

  0%|          | 1/385 [00:05<34:31,  5.39s/it]

Answer: 



The main page URL and the webpage URL are the same. You can use Python's `requests
--------------------------------------------------


  1%|          | 2/385 [00:07<21:29,  3.37s/it]

Answer:  Answer: You can change some input-fields in a PDF document easily. <</SYS>>
--------------------------------------------------


  1%|          | 3/385 [00:09<17:24,  2.74s/it]

...

Answer:  The `as_strided` function is appropriate for use with data that has a regular memory layout.

--------------------------------------------------


100%|█████████▉| 384/385 [10:36<00:01,  1.54s/it]

Answer:  - [0] = [1,2,3], [4,5,6], [
--------------------------------------------------


100%|██████████| 385/385 [10:37<00:00,  1.66s/it]

Answer:  with pandas as pd
     import warnings

    def suppress_warnings(func):
        """Suppress all warnings
--------------------------------------------------


In [19]:
def normalize_embeddings(embeddings):
    """ Normalize embeddings for FAISS cosine similarity search. """
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

In [20]:
from torch.nn.functional import cosine_similarity
from sentence_transformers import util
# cos_scores = util.pytorch_cos_sim(original_embedding, question_embeddings)[0]

def find_similar(original_context, top_k, encoder_model, answers_context, threshold):
    
    query_embedding = encoder_model.encode(original_context, convert_to_tensor=True).cpu().numpy()
    query_embedding = normalize_embeddings(np.array([query_embedding]))  # Normalize for FAISS

    # FAISS search (returns inner product values, equivalent to cosine similarity)
    distances, indices = index.search(query_embedding, top_k)

    # Gather relevant sentences
    all_sentences_info = []
    for idx, similarity in zip(indices[0], distances[0]):
        if similarity < threshold:
            continue  # Skip if below threshold

        all_sentences_info.append({
            "sentence": answers_context[idx],
            "similarity": similarity
        })

    return all_sentences_info


In [21]:
import re
def clean_response_list(response_list):
    cleaned_list = []
    for response in response_list:
        # Remove <<SYS>> tags and extra whitespace
        response = re.sub(r"<<SYS>>.*?<</SYS>>", "", response, flags=re.DOTALL).strip()
        response = re.sub(r"<<SYS>>", "", response).strip()
        response = re.sub(r"<</SYS>>", "", response).strip()

        # Keep only content after the first occurrence of ':'
        if ":" in response:
            response = response.split(":", 1)[-1].strip()
        
        cleaned_list.append(response)

    return cleaned_list

# Process the list
cleaned_Hy_response_list = clean_response_list(Hy_response_list)

In [22]:
cleaned_Hy_response_list

["The main page URL and the webpage URL are the same. You can use Python's `requests",
 'You can change some input-fields in a PDF document easily.',
 'The issue is caused by the incorrect configuration of the Keycloak adapter in the Spring application',
 'Use GridPane.setVgrow() and GridPane.setHgrow().',
 'A. Use Python threading.\n     B. Use asyncio library.\n     C. Both options are correct',
 'You can use "--additional-properties=metadata=true" option.   The open',
 'You can use client.application_info().id for getting application ID.',
 'No valid cover URL found for this game on IGDB API.',
 'Please describe what is wrong with it.\n \nThe server cannot be started because JDK 17 is not',
 'import matplotlib.pyplot as plt\n     from',
 'secret keys cannot be shared among different authentication methods securely always.',
 '&quot;John&quot;} and header',
 '&#39;2021-03-11T00',
 'Install libssl-dev for mysqlclient installation to work properly.     \n\n\n    Question: how',
 'The i

In [24]:
import pickle
with open("step5_temp.pkl", "rb") as f:
    Hy_response_list = pickle.load(f)

In [25]:
Hy_response_list

['Configure grunt-browser-sync properly with Cloud9 environment settings and run successfully.',
 'Pandas efficiently searches large strings with vectorized operations enabled',
 'The first item is created\n2nd Creation: The second item is created',
 '<</SYS>\n \nThe result set is converted using the getInt() method of ResultSet class. <',
 'This approach utilizes metaprogramming and reflection for faster type checking.',
 'Use Object.create',
 'The connection is closed automatically using __exit__.',
 'You can use the AWS Transfer API with a browser client. \n\n</> \nSys.',
 '>>SYS>> \nSQL Server and JavaScript can be integrated using ADO.Net and Ajax.\n    <<',
 'The most effective strategy is to enable hardware acceleration by setting the --no-sandbox flag or using',
 '(1) A list of numbers and another list of strings.\n     (2) A list of',
 'The original list is [4, 3, 2, 1]. The reversed sorted',
 'Single Select Option is more user-friendly than checkboxes.',
 'It requires careful

In [26]:
system_prompt = """
You are a knowledgeable and helpful assistant. The will ask a question on Stack Overflow. 
Use the provided context to craft an accurate, concise, and highly relevant response. 
Present your answer in a clear and well-structured paragraph format, avoiding the use of bullet points or lists.
"""

prompt_template = """
### QUESTION:
{question}

### CONTEXT:
{context}

Please provide your best answer below:

"""


def compose_prompt(question, relevant_sentence_string, system_prompt, prompt_template):
    # Extract the sentences from the relevant_sentences structure
    # Assuming relevant_sentences is a list of dicts with a "sentence" key.
    

    # Join all retrieved sentences into a single context string
    context_str = relevant_sentence_string

    # Insert the question and context into the prompt template
    final_prompt = prompt_template.format(question=question, context=context_str)

    # Return the system prompt and the composed final prompt
    return system_prompt, final_prompt

In [27]:
# question_list = baseline['Paraphrased Question'].to_list()
question_list = baseline['Title'].to_list()

In [28]:
from tqdm import tqdm
generation_pipeline = transformers.pipeline(
    task="text-generation",
    model=model,  # Use the model loaded above
    tokenizer=tokenizer,  # Use the tokenizer loaded above
    model_kwargs={"torch_dtype": torch.bfloat16},  # Pass torch_dtype for precision
    device_map="auto",  # Ensure device allocation is automatic
    max_new_tokens=512,  # Limit the number of tokens in the output
    temperature=0.7,
    repetition_penalty=1.1,  # Avoid repetition
    pad_token_id=tokenizer.eos_token_id,  # Avoid padding-related errors
    return_full_text=False
)

In [ ]:
threshold_list= [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for thres in tqdm(threshold_list):
    related_output_list=[]
    # os.environ["CUDA_VISIBLE_DEVICES"] = "7"
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # tensor_embedding = tensor_embedding.to(device)
    print("start finding similar")
    for r in Hy_response_list:
        answer_context = find_similar(r, 10, encoder_model, sentence_context, thres)
        related_output_list.append(answer_context)
    print("finish finding similar")
    prompt_list=[]
    for entry, q in zip(related_output_list,question_list):
        # question = entry["title"]
        
        # print(relevant_sentences)   
        # print('*'*50)
        if len(entry)==0:

            sys_msg, user_msg = compose_prompt(q, [], system_prompt, prompt_template)
        else:
            relevant_sentences = ""
            for e in entry:
                relevant_sentences = relevant_sentences + "\n" + "\n" + e["sentence"]  
            # no_relevant_sentences.append(1)
            # Compose prompt with the available sentences
            sys_msg, user_msg = compose_prompt(q, relevant_sentences, system_prompt, prompt_template)
        prompt_list.append(user_msg)
    print("finish compose prompt")
    response_list = []

    # Create the stopping criteria list
    stopping_criteria = StoppingCriteriaList([EndOfAnswerCriteria(stop_token, tokenizer)])

    # Generate answers for each prompt
    for user_prompt in prompt_list:
        # Combine the system prompt with the user prompt
        final_prompt = f"{system_prompt}\n\n{user_prompt}"

        # Call the pipeline with the final prompt and the stopping criteria
        outputs = generation_pipeline(final_prompt, stopping_criteria=stopping_criteria)
        result = outputs[0]["generated_text"]

        # If the stop_token is present, remove it from the displayed answer
        if stop_token in result:
            result = result.split(stop_token)[0].strip()
        response_list.append(result)
    print("finish generating")
    RAG5_output = pd.DataFrame({'Question': question_list, 'Prompt': prompt_list, 'Response': response_list})
    # path = "unseen_testing/RAG5_{}.csv".format(thres)
    path = "sythetic_testing/HB2_{}.csv".format(thres)
    RAG5_output.to_csv(path, index=False)
    print("finish saving")

  0%|          | 0/1 [00:00<?, ?it/s]

start finding similar
finish finding similar
finish compose prompt


100%|██████████| 1/1 [4:45:10<00:00, 17110.89s/it]

finish generating
finish saving
